In [6]:
import cv2
import numpy as np
import glob
import os


def create_rectify_maps(K, D, R_rect, P_rect, size):
    return cv2.initUndistortRectifyMap(
        K, D, R_rect, P_rect, size, cv2.CV_32FC1
    )


def rectify_image(img_path, mapx, mapy):
    img = cv2.imread(img_path)
    return cv2.remap(img, mapx, mapy, cv2.INTER_LINEAR)

def draw_horizontal_guides(img, color=(0, 255, 0)):
    h, w = img.shape[:2]
    step = h // 10
    out = img.copy()
    for y in range(0, h, step):
        cv2.line(out, (0, y), (w, y), color, 1)
    return out



In [7]:
# Manualtly entered calibration parameters

#LEFT CAMERA
# Intrinsic matrix (3x3)
K_l = np.array([
    [960.87, 0.0, 681.52],
    [0.0, 960.87, 199.59],
    [0.0, 0.0, 1.0]
])

# Distortion coefficients (5,)
D_l = np.array([
    -0.3641,
    0.1788,
    0.00086,
    -0.00273,
    -0.0420
], dtype=np.float32)


#RIGHT CAMERA
# Intrinsic matrix (3x3)
K_r = np.array([
    [910.54, 0., 676.73],
    [0., 910.54, 214.58],
    [0., 0., 1.]
 ])

# Distortion coefficients (5,)
D_r = np.array([
    -0.38562,
    0.23706,
    0.00110,
    0.00070,
    -0.09438
], dtype=np.float32)


# Rotation matrix (3x3)
R = np.array([
    [0.9993, 0.0243, -0.0271],
    [-0.0246, 0.9996, -0.0107],
    [0.0268, 0.0113, 0.9996]
])

# Translation vector (3x1) left to right
T = np.array([
    [-0.4692],
    [-0.0106],
    [0.0308]
], dtype=np.float32)


image_size = (1392, 512) # Width, Height



In [ ]:
# Change these paths

# Input directories for left and right images (Have to change seq_xx for different sequences)
input_left = r"C:\Users\bjarn\Desktop\Perception_for_autonomous_systems\34759_final_project_raw\seq_03\image_02\data"
input_right = r"C:\Users\bjarn\Desktop\Perception_for_autonomous_systems\34759_final_project_raw\seq_03\image_03\data"

# Remember to create these output directories beforehand
# Output directories for rectified images (have to change seq_xx for different sequences)
out_left = r"C:\Users\bjarn\Desktop\Perception_for_autonomous_systems\rect_eval\seq_3\left"
out_right = r"C:\Users\bjarn\Desktop\Perception_for_autonomous_systems\rect_eval\seq_3\right"


os.makedirs(out_left, exist_ok=True)
os.makedirs(out_right, exist_ok=True)


R_l, R_r, P_l, P_r, Q, _, _ = cv2.stereoRectify(
    K_l, D_l, K_r, D_r, image_size, R, T, alpha=0
)

# ---- Build rectification maps ----
left_map_x, left_map_y = create_rectify_maps(K_l, D_l, R_l, P_l, image_size)
right_map_x, right_map_y = create_rectify_maps(K_r, D_r, R_r, P_r, image_size)

# ---- Process all images ----
left_images = sorted(glob.glob(os.path.join(input_left, "*.png")))
right_images = sorted(glob.glob(os.path.join(input_right, "*.png")))

for L, R in zip(left_images, right_images):
    frame_id = os.path.basename(L)

    rect_L = rectify_image(L, left_map_x, left_map_y)
    rect_R = rectify_image(R, right_map_x, right_map_y)

    cv2.imwrite(os.path.join(out_left, frame_id), rect_L)
    cv2.imwrite(os.path.join(out_right, frame_id), rect_R)


